In [99]:
import requests
import bs4
import js2py
from IPython.display import JSON

In [83]:
!pip install v8py 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for v8py
Failed to build v8py
    Running setup.py install for v8py: started
    Running setup.py install for v8py: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Zac\AppData\Local\Programs\Python\Python38\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Zac\\AppData\\Local\\Temp\\pip-install-zfob__gt\\v8py_b9ea4bafbc054be3b56e4e87075398e6\\setup.py'"'"'; __file__='"'"'C:\\Users\\Zac\\AppData\\Local\\Temp\\pip-install-zfob__gt\\v8py_b9ea4bafbc054be3b56e4e87075398e6\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Zac\AppData\Local\Temp\pip-wheel-m8jz2k0c'
       cwd: C:\Users\Zac\AppData\Local\Temp\pip-install-zfob__gt\v8py_b9ea4bafbc054be3b56e4e87075398e6\
  Complete output (471 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd6

## Extracting Academy Cinemas Data

In [3]:
# Request homepage
page = requests.get('https://www.academycinemas.co.nz/#')
# Parse HTML
soup = bs4.BeautifulSoup(page.content)
# Extract the script we need
data = soup.find_all('script')[13].text[43:]

In [105]:
context = js2py.EvalJs({})

In [106]:
context.execute(data)

In [115]:
parsed_releases = context.movieData.to_dict()

In [119]:
JSON(parsed_releases)

<IPython.core.display.JSON object>

---

### Building a Calendar

In [149]:
from ics import Calendar, Event
from datetime import datetime, timedelta, timezone
import pytz

In [133]:
datetime.strptime("2022-06-16 6:00pm", "%Y-%m-%d %H:%M%p")

datetime.datetime(2022, 6, 16, 6, 0)

In [163]:
timezone = pytz.timezone('Pacific/Auckland')

In [179]:
def build_calendar(parsed_releases):
    cal = Calendar()
    
    for date, films in parsed_releases.items():
        for film in films:
            for time in film['times']:
                #print(film)
                start_time = datetime.strptime(f"{date} {time['time']}", "%Y-%m-%d %H:%M%p")
                start_time.replace(tzinfo=timezone)
                duration = timedelta(minutes=int(film['duration'][:-5]))
                event = Event(
                    name=film['title'],
                    begin=start_time,
                    end=f"{start_time + duration}",
                    location=f"Academy Cinemas", # todo
                    url=time['bookingLink'],
                    categories=[f"FILM"],
                    description=time['bookingLink']
                )
                cal.events.add(event)
                
                #return cal

    return cal

cal = build_calendar(parsed_releases)

In [180]:
next(iter(cal.events))

<Event 'Whina' begin:2022-06-30T06:00:00+00:00 end:2022-06-30T07:52:00+00:00>

In [181]:
print(next(iter(cal.events)).end.astimezone(timezone))

2022-06-30 19:52:00+12:00


In [182]:
with open('academy_test_calendar.ics', 'w') as f:
    f.write(str(cal))